---

CARGA DE LIBRERÍA Y DATOS

---

In [43]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [44]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score, classification_report
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.stats import chi2
from sklearn.pipeline import Pipeline
from sklearn.impute import KNNImputer
from sklearn.preprocessing import FunctionTransformer, RobustScaler
from sklearn.metrics import mean_squared_error
from sklearn.base import BaseEstimator, TransformerMixin

In [45]:
df = pd.read_csv('/content/drive/MyDrive/Cupido_IA_project/train.csv')
test = pd.read_csv("/content/drive/MyDrive/Cupido_IA_project/test.csv")
submission = pd.read_csv("/content/drive/MyDrive/Cupido_IA_project/sample_submission.csv")

---

## DEFINICIÓN PREPROCESADO

---

In [46]:
cols_to_int = ['age', 'sex', 'cp', 'restecg']

rename_dict = {
    "age": "edad",
    "sex": "sexo",
    "cp": "tipo_dolor_pecho",
    "trestbps": "tension_en_descanso",
    "chol": "colesterol",
    "fbs": "azucar",
    "restecg": "electro_en_descanso",
    "thalach": "latidos_por_minuto",
    "exang": "dolor_pecho_con_ejercicio",
    "oldpeak": "cambio_linea_corazon_ejercicio",
    "slope": "forma_linea_corazon_ejercicio",
    "ca": "num_venas_grandes",
    "thal": "estado_corazon_thal" }

cols_a_clippear = [
    'tension_en_descanso', 'colesterol',
    'latidos_por_minuto', 'cambio_linea_corazon_ejercicio']

categorical_cols_to_round = [
    'num_venas_grandes', 'estado_corazon_thal', 'sexo',
    'tipo_dolor_pecho', 'dolor_pecho_con_ejercicio',
    'azucar', 'forma_linea_corazon_ejercicio', 'electro_en_descanso']

---

FUNCIONES DE PREPROCESADO

---

In [47]:
def limpieza_inicial(df):
    """
    Realiza conversiones de tipos, renombres y limpieza básica de errores (-9).
    Se puede aplicar a todo el dataset antes del split.
    """
    df = df.copy()

    # Conversión a Int
    for col in cols_to_int:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce').astype('Int64')

    # Conversión de objetos a numérico
    object_cols = df.select_dtypes(include=['object']).columns
    for col in object_cols:
        df[col] = pd.to_numeric(df[col], errors='coerce')

    # Renombrar
    df = df.rename(columns=rename_dict)

    # Reemplazar errores conocidos (-9) por NaN
    df.replace([-9, -9.0], np.nan, inplace=True)

    return df

In [48]:
def limpiar_ceros_fisiologicos(X):
    X = X.copy()
    cols_imposibles_con_cero = ['tension_en_descanso', 'colesterol']
    for col in cols_imposibles_con_cero:
        if col in X.columns:
            X[col] = X[col].replace({0: np.nan, 0.0: np.nan})
    return X

In [49]:
def clipear_outliers(X):
    """
    Nota: Usado en FunctionTransformer, esto calculará los cuantiles
    sobre el lote actual de datos.
    """
    X = X.copy()
    for col in cols_a_clippear:
        if col in X.columns:
            p1 = X[col].quantile(0.01)
            p99 = X[col].quantile(0.99)
            X[col] = X[col].clip(lower=p1, upper=p99)
    return X

In [50]:
def crear_flags_mnar(df):
    df_new = df.copy()
    cols_mnar = ['num_venas_grandes', 'estado_corazon_thal']
    for col in cols_mnar:
        if col in df_new.columns:
            df_new[f'{col}_is_missing'] = df_new[col].isna().astype(int)
    return df_new

In [51]:
class RobustKNNImputerWrapper(BaseEstimator, TransformerMixin):
    """
    1. Escala los datos (RobustScaler) para que KNN funcione bien por distancias.
    2. Imputa los valores.
    3. Invierte el escalado para devolver los datos en unidades ORIGINALES.
    """
    def __init__(self, n_neighbors=5):
        self.n_neighbors = n_neighbors
        self.scaler = RobustScaler()
        self.imputer = KNNImputer(n_neighbors=n_neighbors, weights='distance')
        self.feature_names_in_ = None
        super().__init__()

    def fit(self, X, y=None):
        # Guardamos nombres de columnas para reconstruir el DF después
        self.feature_names_in_ = X.columns if hasattr(X, 'columns') else [f"feat_{i}" for i in range(X.shape[1])]

        # Ajustamos el scaler y el imputer
        X_scaled = self.scaler.fit_transform(X)
        self.imputer.fit(X_scaled)
        return self

    def transform(self, X):
        X_scaled = self.scaler.transform(X)
        X_imputed_scaled = self.imputer.transform(X_scaled)
        # 3. Invertir escalado (volver a magnitud original)
        X_imputed = self.scaler.inverse_transform(X_imputed_scaled)
        return pd.DataFrame(X_imputed, columns=self.feature_names_in_, index=X.index)

    def set_output(self, *, transform=None):
        return self

In [52]:
def aplicar_feature_engineering_avanzado(df):
    """
    Aplica transformaciones basadas en evidencia no lineal y combinaciones Tier 1.
    Asume que los datos entran imputados y en escala original.
    """
    df = df.copy()

    # 1. Transformación No-Lineal: Oldpeak (Flag Binario)
    # Evidencia: Comportamiento de "interruptor". Si hay depresión ST, el riesgo se dispara.
    if 'cambio_linea_corazon_ejercicio' in df.columns:
        df['flag_depresion_st'] = (df['cambio_linea_corazon_ejercicio'] > 0).astype(int)

    # 2. Transformación No-Lineal: Colesterol (Binning)
    # Evidencia: Forma de campana asimétrica. Elimina ruido numérico de extremos.
    if 'colesterol' in df.columns:
        conditions = [
            (df['colesterol'] < 200),
            (df['colesterol'] >= 200) & (df['colesterol'] < 240),
            (df['colesterol'] >= 240)
        ]
        choices = [0, 1, 2] # Normal, Limitrofe, Alto
        df['rango_colesterol'] = np.select(conditions, choices, default=0)

    # 3. Transformación No-Lineal: Tensión (Umbral Hipertensión)
    # Evidencia: Efecto umbral, solo importa si es alta.
    if 'tension_en_descanso' in df.columns:
        df['flag_hipertension'] = (df['tension_en_descanso'] > 130).astype(int)

    # 4. Interacción (Tier 1): Capacidad Cardíaca Real
    # Normaliza las pulsaciones por la edad (220 - edad = max teórico).
    if 'latidos_por_minuto' in df.columns and 'edad' in df.columns:
        max_hr = 220 - df['edad']
        # Evitar división por cero (aunque edad > 220 es imposible)
        df['porcentaje_frecuencia_max'] = df['latidos_por_minuto'] / max_hr.replace(0, 1)

    # 5. Interacción (Tier 1): Score de Respuesta al Estrés
    # Suma de angina inducida + depresión ST.
    if 'dolor_pecho_con_ejercicio' in df.columns and 'flag_depresion_st' in df.columns:
        df['score_respuesta_stress'] = df['dolor_pecho_con_ejercicio'] + df['flag_depresion_st']

    # 6. Agregación (Tier 3): Carga de Comorbilidad
    # Agrupa variables débiles (azúcar + hipertensión + electro basal)
    cols_comorbilidad = ['azucar', 'flag_hipertension', 'electro_en_descanso']
    if set(cols_comorbilidad).issubset(df.columns):
        # Aseguramos que electro > 0 cuente como 1 punto
        electro_punto = (df['electro_en_descanso'] > 0).astype(int)
        df['carga_comorbilidad'] = df['azucar'] + df['flag_hipertension'] + electro_punto

    return df

In [53]:
def redondear_imputaciones(X):
    X = X.copy()
    for col in categorical_cols_to_round:
        if col in X.columns:
            X[col] = X[col].round()
    return X

In [54]:
def optimizar_k_knn(X_train, k_range=[3, 5, 7, 9, 11, 15]):
    # Preparamos una copia limpia para testear
    X_temp = limpiar_ceros_fisiologicos(X_train)
    X_temp = clipear_outliers(X_temp)
    # Solo usamos filas completas para validar el error de imputación
    X_complete = X_temp.dropna().copy()

    if len(X_complete) < 50:
        print("Pocos datos completos. Se usará k=5 por defecto.")
        return 5

    rmse_scores = {}
    scaler = RobustScaler()
    X_scaled_array = scaler.fit_transform(X_complete)

    np.random.seed(42)
    mask = np.random.rand(*X_scaled_array.shape) < 0.1
    X_missing_sim = X_scaled_array.copy()
    X_missing_sim[mask] = np.nan

    print(f"Buscando k óptimo sobre {len(X_complete)} muestras...")
    for k in k_range:
        imputer = KNNImputer(n_neighbors=k, weights='distance')
        X_imputed = imputer.fit_transform(X_missing_sim)
        error = np.sqrt(mean_squared_error(X_scaled_array[mask], X_imputed[mask]))
        rmse_scores[k] = error

    best_k = min(rmse_scores, key=rmse_scores.get)
    print(f"Mejor k encontrado: {best_k}")
    return best_k

---
## APLICACIÓN DEL FLUJO DE PREPROCESADO DEFINIDO

---

In [55]:
df_train = df.copy()
df_test = test.copy()

df_train = limpieza_inicial(df_train)
df_test = limpieza_inicial(df_test)

target = "label"

X_train = df_train.drop(columns=target)
y_train = df_train[target]

if target in df_test.columns:
    X_test = df_test.drop(columns=target)
    y_test = df_test[target]
else:
    X_test = df_test.copy()

best_k = optimizar_k_knn(X_train)


pipeline_feature_engineering = Pipeline([
    ('limpieza_ceros', FunctionTransformer(limpiar_ceros_fisiologicos, validate=False)),
    ('clipear_outliers', FunctionTransformer(clipear_outliers, validate=False)),
    ('mnar_flags', FunctionTransformer(crear_flags_mnar, validate=False)),
    ('imputacion_robusta', RobustKNNImputerWrapper(n_neighbors=15)),
    ('rounding', FunctionTransformer(redondear_imputaciones, validate=False)),
    ('feature_engineering', FunctionTransformer(aplicar_feature_engineering_avanzado, validate=False)),
    ('final_scaler', RobustScaler())]).set_output(transform="pandas")

Buscando k óptimo sobre 246 muestras...
Mejor k encontrado: 15


---
## EJECUCIÓN Y VERIFICACIÓN DE TRANSFORMACIÓN

---

In [56]:
print("Ajustando pipeline completo con Feature Engineering...")
X_train_prep = pipeline_feature_engineering.fit_transform(X_train)
y_train_prep = y_train.copy()

print("Aplicando transformaciones a TEST...")
X_test_prep = pipeline_feature_engineering.transform(X_test)

print("\n--- Proceso finalizado ---")
print(f"Dimensiones Train final (X_train_prep): {X_train_prep.shape}")
print(f"Nulos restantes: {X_train_prep.isna().sum().sum()}")

Ajustando pipeline completo con Feature Engineering...
Aplicando transformaciones a TEST...

--- Proceso finalizado ---
Dimensiones Train final (X_train_prep): (732, 21)
Nulos restantes: 0


---

## VALIDACIÓN DEL PODER PREDICTIVO DE LAS NUEVAS VARIABLES

---

---
VARIABLES CUANTITATIVAS

---

In [57]:
df_analisis = X_train_prep.copy()
df_analisis['label'] = y_train_prep.values

nuevas_cols_cont = ['edad', 'tension_en_descanso', 'colesterol',
    'latidos_por_minuto', 'cambio_linea_corazon_ejercicio',
    # Nuevas variables creadas
    'porcentaje_frecuencia_max']

resultados_anova = []
print("="*50)
print("📊 ANÁLISIS ANOVA (Variables Continuas)")

for col in nuevas_cols_cont:
    if col in df_analisis.columns:
        groups = [
            df_analisis[col][df_analisis['label'] == level]
            for level in df_analisis['label'].unique()]
        f_statistic, p_value = f_oneway(*groups)

        resultados_anova.append({
            "Variable": col,
            "F_Score": round(f_statistic, 2),
            "P_Value": round(p_value, 5) })

df_anova = pd.DataFrame(resultados_anova).sort_values(by="F_Score", ascending=False)
display(df_anova)

📊 ANÁLISIS ANOVA (Variables Continuas)


,Variable,F_Score,P_Value
4,cambio_linea_corazon_ejercicio,56.08,0.00000
3,latidos_por_minuto,39.89,0.00000
5,porcentaje_frecuencia_max,26.06,0.00000
0,edad,18.10,0.00000
1,tension_en_descanso,5.58,0.00020
2,colesterol,3.12,0.01479


---

VARIABLES CATEGÓRICAS

---

In [58]:

nuevas_cols_cat = ['sexo', 'tipo_dolor_pecho', 'azucar', 'electro_en_descanso',
    'dolor_pecho_con_ejercicio', 'forma_linea_corazon_ejercicio',
    'num_venas_grandes', 'estado_corazon_thal',
    # Nuevas variables creadas
    'num_venas_grandes_is_missing', 'estado_corazon_thal_is_missing',
    'flag_depresion_st', 'flag_hipertension', 'rango_colesterol',
    'score_respuesta_stress', 'carga_comorbilidad' ]

def cramers_v_safe(x, y):

    x_cat = x.round().astype(str)
    y_cat = y.astype(str)

    cont = pd.crosstab(x_cat, y_cat)
    if cont.size == 0: return 0.0

    chi2, _, _, _ = chi2_contingency(cont)
    n = cont.sum().sum()
    phi2 = chi2 / n
    r, k = cont.shape

    if n <= 1: return 0.0
    phi2_corr = max(0, phi2 - (k-1)*(r-1)/(n-1))
    r_corr = r - (r-1)**2/(n-1)
    k_corr = k - (k-1)**2/(n-1)

    denom = min((k_corr-1), (r_corr-1))
    if denom <= 0: return 0.0
    return np.sqrt(phi2_corr / denom)

print("\n" + "="*50)
print("📊 ANÁLISIS CRAMÉR'S V (Variables Categóricas/Flags)")

results_cramer = []
for col in nuevas_cols_cat:
    if col in df_analisis.columns:
        score = cramers_v_safe(df_analisis[col], df_analisis['label'])
        results_cramer.append({
            "Variable": col,
            "Asociación (V)": score})

df_cramer = pd.DataFrame(results_cramer).sort_values(by="Asociación (V)", ascending=False)
display(df_cramer.style.background_gradient(cmap="Greens", subset=["Asociación (V)"]).format({"Asociación (V)": "{:.3f}"}))


📊 ANÁLISIS CRAMÉR'S V (Variables Categóricas/Flags)


,Variable,Asociación (V)
4,dolor_pecho_con_ejercicio,0.448
13,score_respuesta_stress,0.422
7,estado_corazon_thal,0.389
10,flag_depresion_st,0.362
6,num_venas_grandes,0.312
0,sexo,0.305
1,tipo_dolor_pecho,0.289
5,forma_linea_corazon_ejercicio,0.223
11,flag_hipertension,0.147
14,carga_comorbilidad,0.132


# Propuesta de estrategia de eliminación agresiva


### 🔴 GRUPO 1: Eliminación Directa (Ruido Puro)
Estas variables tienen una asociación casi nula ($V < 0.1$ o $F$ muy bajo). Solo aportan confusión al modelo.

* **`electro_en_descanso` ($V = 0.069$):** Aporta menos información que tirar una moneda.
* **`estado_corazon_thal_is_missing` ($V = 0.040$):** Al contrario que las venas, que falte el dato de 'thal' no predice nada.
* **`rango_colesterol` y `colesterol`:** El continuo tiene un $F$ bajo (3.12) y el rango categórico un $V$ bajísimo (0.089). En este dataset, el colesterol no sirve para predecir el grado de enfermedad.

---

### 🟠 GRUPO 2: Eliminación por Redundancia (Absorción)
Variables débiles que ya hemos "salvado" metiéndolas en una variable combinada más fuerte. Al eliminar las originales, reducimos dimensiones sin perder información.

* **`azucar` ($V = 0.13$):** Es débil. Ya la hemos incluido dentro de `carga_comorbilidad`. Quédate con la carga, elimina el azúcar suelto.
* **`tension_en_descanso`:** La variable continua es ruidosa ($F=5.58$). La `flag_hipertension` ($V=0.14$) captura mejor el riesgo, y la `carga_comorbilidad` también la incluye. Elimina la continua.
* **`porcentaje_frecuencia_max`:** Tu análisis demostró que la variable original `latidos_por_minuto` ($F=39.89$) es mucho más potente que la calculada ($F=26.06$). Elimina la calculada, quédate la original.

---

### 🟢 GRUPO 3: El "Core" Predictivo (Intocables)
Estas son las variables que sostendrán tu F1 Score.

* **Drivers:** `dolor_pecho_con_ejercicio`, `score_respuesta_stress`, `estado_corazon_thal`, `latidos_por_minuto`, `cambio_linea_corazon_ejercicio` (o su flag).
* **Demográficos:** `sexo`, `edad`.
* **Anatómicos:** `num_venas_grandes` y `num_venas_grandes_is_missing`.